# 1. Symplectic representation

## 1.1. On Humphries generators

### Sp_repr class and functions

In [1]:
%display latex

In [2]:
class Sp_repr:
    #--- 2x2 ---
    I = identity_matrix(2)
    O = zero_matrix(ZZ,2)
    J = matrix([[0,1],[-1,0]])
    #---
    L = matrix([[1,0],[1,1]])
    K = matrix([[0,-1],[0,0]])
    #--- 4x4 ---
    J4 = block_matrix([[J, O], [O, J]])
    # J4 = J4_bm.subs([(J, evJ)]).as_explicit()
    #---
    CHR_2_MTX = {'a': block_matrix([[L.inverse(), O], [O, I]]),
                 'b': block_matrix([[L.T, K], [K, L.T]]),
                 'c': block_matrix([[I, O], [O, L.inverse()]]),
                 'd': block_matrix([[I, O], [O, L.T]]),
                 'f': block_matrix([[L.T, O], [O,I]])}
    #---
    
    def __init__(self, l:str):
        self.loop = l
        self.matrix = self.CHR_2_MTX.get(l) if l.islower() else self.CHR_2_MTX.get(l.lower()).inverse()

In [3]:
def is_Sp(M):
    J = Sp_repr.J4
    return M.transpose()*J*M == J

list(map(lambda l: is_Sp(Sp_repr(l).matrix), ['a','b','c','d','f', 'A'])) 

[True, True, True, True, True, True]

## 1.2. The symplectic images of Abcd and abcD

In [4]:
a,b,c,d,f = tuple(map(lambda l: Sp_repr(l).matrix, ['a','b','c','d','f']))
J4 = Sp_repr.J4

a,b,c,d,f, J4 

(
[ 1  0| 0  0]  [ 1  1| 0 -1]  [ 1  0| 0  0]  [1 0|0 0]  [1 1|0 0]
[-1  1| 0  0]  [ 0  1| 0  0]  [ 0  1| 0  0]  [0 1|0 0]  [0 1|0 0]
[-----+-----]  [-----+-----]  [-----+-----]  [---+---]  [---+---]
[ 0  0| 1  0]  [ 0 -1| 1  1]  [ 0  0| 1  0]  [0 0|1 1]  [0 0|1 0]
[ 0  0| 0  1], [ 0  0| 0  1], [ 0  0|-1  1], [0 0|0 1], [0 0|0 1],

[ 0  1| 0  0]
[-1  0| 0  0]
[-----+-----]
[ 0  0| 0  1]
[ 0  0|-1  0]
)

In [5]:
A, B, C, D, F = (k.inverse() for k in [a,b,c,d,f])
A,B,C,D,F

(
[1 0 0 0]  [ 1 -1  0  1]  [1 0 0 0]  [ 1  0  0  0]  [ 1 -1  0  0]
[1 1 0 0]  [ 0  1  0  0]  [0 1 0 0]  [ 0  1  0  0]  [ 0  1  0  0]
[0 0 1 0]  [ 0  1  1 -1]  [0 0 1 0]  [ 0  0  1 -1]  [ 0  0  1  0]
[0 0 0 1], [ 0  0  0  1], [0 0 1 1], [ 0  0  0  1], [ 0  0  0  1]
)

In [6]:
M = a*b*c*D
display(M)
N = A*b*c*d
display(N)

list(map(is_Sp, [M, N]))

[ 1  1  1 -2]
[-1  0 -1  2]
[ 0 -1  0  1]
[ 0  0 -1  2]

[ 1  1  1  0]
[ 1  2  1  0]
[ 0 -1  0  1]
[ 0  0 -1  0]

[True, True]

### The charactoristic polynomial of M (and N)

In [7]:
R.<a> = PolynomialRing(ZZ)
cp_M = M.charpoly('t'); display(cp_M, cp_M.factor())
cp_N = N.charpoly('t'); #display(cp_N)

print(f"cp_M == cp_N ?   --> {cp_M == cp_N}")

t^4 - 3*t^3 + 3*t^2 - 3*t + 1

t^4 - 3*t^3 + 3*t^2 - 3*t + 1

cp_M == cp_N ?   --> True


# 2. The ideal for fibered knots

In [8]:
K.<a> = NumberField(cp_M)
OK = K.ring_of_integers()
print("整数基:", OK.basis())
print("類数:", OK.class_number())

U = UnitGroup(K)
print(U.gens_values())

G = K.galois_group()
print(G)

整数基: [1, a, a^2, a^3]
類数: 1
[-1, a - 1, -a^3 + 3*a^2 - 3*a + 2]
Galois group 4T3 (D(4)) with order 8 of t^4 - 3*t^3 + 3*t^2 - 3*t + 1


In [9]:
def ev_wrt_a(X):
    X_K = X.change_ring(K)
    evlist = X_K.eigenvectors_right();
    v = [v[1][0] for v in evlist if v[0] == a][0]
    if X_K*v == a*v:
        return v
    else:
        return None

### Eigenvectors of M and N with respect to the eigen value $a$

In [93]:
v, w = [ev_wrt_a(X) for X in (M,N)]
display(v, w)
print("------- Check -------")
print(M*v == a*v, N*w == a*w)

(1, -a^3 + 2*a^2 - a + 1, -3*a^3 + 8*a^2 - 6*a + 6, -2*a^3 + 5*a^2 - 4*a + 4)

(1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a)

------- Check -------
True True


### Ideals of M and N

In [95]:
# The integral ideal (v) is equal to ZZ[a]

T = matrix([[1,0,0,0],[0,1,0,0],[0,0,-1,1],[0,0,1,-2]]); display(T, T*v)
T2 = matrix([[1,0,0,0],[0,1,0,1],[0,0,1,-1],[0,0,0,1]]); display(T2*(T*v))
T3 = matrix([[1,0,0,0],[1,1,0,0],[0,0,-1,0],[0,-2,-2,1]]); display(T3*(T2*(T*v)))
X = T3*T2*T; display(X)

[ 1  0  0  0]
[ 0  1  0  0]
[ 0  0 -1  1]
[ 0  0  1 -2]

(1, -a^3 + 2*a^2 - a + 1, a^3 - 3*a^2 + 2*a - 2, a^3 - 2*a^2 + 2*a - 2)

(1, a - 1, -a^2, a^3 - 2*a^2 + 2*a - 2)

(1, a, a^2, a^3)

[ 1  0  0  0]
[ 1  1  1 -2]
[ 0  0  2 -3]
[ 0 -2  3 -4]

In [96]:
# The integral ideal (w) is equal to ZZ[a]

T = matrix([[1,0,0,0],[0,1,1,0],[0,0,0,1],[0,0,1,0]]); display(T, T*w)
T2 = matrix([[1,0,0,0],[1,1,0,0],[2,2,1,0],[4,4,2,-1]]); display(T*w, T2*(T*w))
Y = T2*T; display(Y)

[1 0 0 0]
[0 1 1 0]
[0 0 0 1]
[0 0 1 0]

(1, a - 1, a^2 - 2*a, -a^3 + 2*a^2)

(1, a - 1, a^2 - 2*a, -a^3 + 2*a^2)

(1, a, a^2, a^3)

[1 0 0 0]
[1 1 1 0]
[2 2 2 1]
[4 4 3 2]

In [122]:
S = Y.inverse()*X
display(S, S.inverse())

print(f"{(S*v == w) = }")
display(S*v, w)
print(f"{is_Sp(S)=}")

[ 1  0  0  0]
[ 0 -1  0  0]
[ 0  2  1 -2]
[-2 -2  0  1]

[ 1  0  0  0]
[ 0 -1  0  0]
[ 4 -2  1  2]
[ 2 -2  0  1]

(S*v == w) = True


(1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a)

(1, a^3 - 2*a^2 + a - 1, -a^3 + 2*a^2, a^2 - 2*a)

is_Sp(S)=False


### Scratch

In [22]:
T = matrix([[-1,0,-2,-2],[0,1,2,0],[0,0,1,2],[0,0,0,1]])
display(T)
is_Sp(T)

[-1  0 -2 -2]
[ 0  1  2  0]
[ 0  0  1  2]
[ 0  0  0  1]

False

In [15]:
v = vector([a^3-2*a^2+2*a-2, a^2+1, -a, 1])
w = vector([a^2*(2-a), (a-1)^2, -(a-2), 1])
display(v,w, T*v, T*v == w)

(a^3 - 2*a^2 + 2*a - 2, a^2 + 1, -a, 1)

(-a^3 + 2*a^2, a^2 - 2*a + 1, -a + 2, 1)

(-a^3 + 2*a^2, a^2 - 2*a + 1, -a + 2, 1)

True

In [16]:
display(N*w, a*w)

(-a^3 + 3*a^2 - 3*a + 3, -a^3 + 4*a^2 - 5*a + 4, -a^2 + 2*a, a - 2)

(-a^3 + 3*a^2 - 3*a + 1, a^3 - 2*a^2 + a, -a^2 + 2*a, a)

In [ ]:
#display(X.inverse()*M*X)